Load in variables and prep

In [106]:
# import vars and dandi api
import numpy as np
from dandi.dandiapi import DandiAPIClient
import os # import os

# define dataset
dandiset_id = '000067' # dataset

# get dataset
dandiset = client.get_dandiset(dandiset_id,'draft')

# list splits the "zipped" generator file
assets   = list(dandiset.get_assets())

# list comprehension
rats = {os.path.split(x.path)[0] for x in assets} # list comprehension, for loop, {} if unique
#print(rats)
#print(assets)

# a set of sessions
#sessions = {os.path.split(x.path) for x in assets}
#print(sessions)

# if session belongs to rat(i), then store data in dict

# actually just work with sessions for now and call it a day
sessions = [x.path for x in assets]
print(sessions)

['sub-EE/sub-EE_ses-EE-044_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-043_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-081_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-080_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-082_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-083_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-085_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-042_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-046_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-086_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-087_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-084_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-088_behavior+ecephys.nwb', 'sub-EE/sub-EE_ses-EE-089_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-047_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-048_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-049_behavior+ecephys.nwb', 'sub-EE/sub-EE_ses-EE-050_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-045_ecephys.nwb', 'sub-EE/sub-EE_ses-EE-196_behavior+ecephys.nwb', 'sub-EE/sub-EE_ses-EE-210_ecephys.nwb', 'sub-FF/sub-FF_ses-FF-167_behavior+ecephys.nwb', 'sub-FF/sub-FF_ses-FF-130_behavior+ecephys.nwb', 'sub-FF/sub-FF_ses-FF-103_behavior

In [107]:
#for sessi in len(sessions):
sessi = 1
filepath = sessions[0]

with DandiAPIClient() as client:
    dataAsset = client.get_dandiset(dandiset_id,'draft').get_asset_by_path(filepath)
    s3_url    = asset.get_content_url(follow_redirects=1, strip_query=True)

# now we have the s3 url for the first dataset to work with
print(s3_url)

https://dandiarchive.s3.amazonaws.com/blobs/43b/f3a/43bf3a81-4a0b-433f-b471-1f10303f9d35


### Define some stuff prior to analyzing

In [108]:
import pynwb
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
import requests

def _search_assets(url, filepath):
    response = requests.request("GET", url, headers={"Accept": "application/json"}).json() 
    
    for asset in response["results"]:
        if filepath == asset["path"]:
            return asset["asset_id"]
    
    if response.get("next", None):
        return _search_assets(response["next"], filepath)
    
    raise ValueError(f'path {filepath} not found in dandiset {dandiset_id}.')


def get_asset_id(dandiset_id, filepath):
    url = f"https://api.dandiarchive.org/api/dandisets/{dandiset_id}/versions/draft/assets/"
    return _search_assets(url, filepath)

def get_s3_url(dandiset_id, filepath):
    """Get the s3 location for any NWB file on DANDI"""

    asset_id = get_asset_id(dandiset_id, filepath)
    url = f"https://api.dandiarchive.org/api/dandisets/{dandiset_id}/versions/draft/assets/{asset_id}/download/"
    
    s3_url = requests.request(url=url, method='head').url
    if '?' in s3_url:
        return s3_url[:s3_url.index('?')]
    return s3_url

ModuleNotFoundError: No module named 'nwbwidgets'